In [2]:
from sklearn.model_selection import train_test_split
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

image_data = []
labels = []

for dirname, _, filenames in os.walk('/kaggle/input/sharks-kangaroo-dataset'):
    for filename in filenames:
        img = cv.imread(os.path.join(dirname, filename))
        image_data.append(img)
        
        # Parse the label from the filename and add it to the labels list
        label = filename.split('_')[0]
        if label[0]=='k':
            binary_label = 1
        else:
            binary_label = 0
        labels.append(binary_label)


image_data = np.array(image_data)
labels = np.array(labels)

# print(image_data.shape)
print(labels)

# Splitting into training and test data
train_data, test_data, train_labels, test_labels = train_test_split(image_data, labels, test_size=0.2, random_state=42)

# split the training set further into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

# print(type(train_labels))
# print(train_data.shape)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.metrics import *
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard
import time

In [40]:
num_epochs = 5
batch_size = 32

In [41]:
# Define the model architecture: VGG16(Block1)
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [42]:
# compile the model
# sgd = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [43]:
# create a new TensorBoard callback
log_dir = "./logs"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# train the model
t1 = time.time()
model.fit(train_data, train_labels, epochs=num_epochs, validation_data=(val_data, val_labels), callbacks=[tensorboard_callback])
t2 = time.time()
print('Time taken in training:',t2-t1,'seconds')

Epoch 1/5
4/4 [==============================] - ETA: 0s - loss: 9881.9961 - accuracy: 0.5078 

In [ ]:
# # Run this command to open tensorboard in a new tab

# %load_ext tensorboard
# %tensorboard --logdir ./logs --port=8080


/bin/bash: line 0: kill: (112) - No such process


In [21]:
# evaluate the model on the test set
loss, acc = model.evaluate(test_data, test_labels)

print('Test Loss: {:.2f}, Test Accuracy: {:.2f}%'.format(loss, acc*100))
print(model.summary())

2/2 [==============================] - 7s 1s/step - loss: 131.3493 - accuracy: 0.4750
Test Loss: 131.35, Test Accuracy: 47.50%
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_12 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_14 (Conv2D)          (None, 112, 112, 128)     147584    
                                                           

#### Using 3-block VGG

In [22]:
# create a model
model = Sequential()

# Block 1
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(224,224,3)))
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

# Block 2
model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

# Block 3
model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
t1 = time.time()
model.fit(train_data, train_labels, epochs=num_epochs, validation_data=(val_data, val_labels))
t2 = time.time()
print('Time take in training:',t2-t1,'seconds')

4/4 [==============================] - 83s 21s/step - loss: 645.2166 - accuracy: 0.4531 - val_loss: 12.5941 - val_accuracy: 0.4688
Time take in training: 83.8485860824585 seconds


In [23]:
# evaluate the model on the test set
loss, acc = model.evaluate(test_data, test_labels)

print('Test Loss: {:.2f}, Test Accuracy: {:.2f}%'.format(loss, acc*100))
print(model.summary())

2/2 [==============================] - 7s 1s/step - loss: 12.3173 - accuracy: 0.5250
Test Loss: 12.32, Test Accuracy: 52.50%
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_19 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_20 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_21 (Conv2D)          (None, 112, 112, 128)     147584    
                                                             

# Using VGG(3-Blocks) with Data Augmentation

In [24]:
# define the data augmentation generator
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [25]:
# train the model with data augmentation
t1 = time.time()
model.fit(
    train_datagen.flow(train_data, train_labels, batch_size=batch_size),
    steps_per_epoch=len(train_data) // batch_size,
    epochs=num_epochs,
    validation_data=(val_data, val_labels),
    validation_steps=len(val_data) // batch_size)

t2 = time.time()
print('Time take in training:',t2-t1,'seconds')

4/4 [==============================] - 89s 22s/step - loss: 5.4889 - accuracy: 0.4844 - val_loss: 1.6311 - val_accuracy: 0.5000
Time take in training: 89.71563291549683 seconds


In [26]:
# evaluate the model on the test data
loss, acc = model.evaluate(test_data, test_labels)
print('Test Loss: {:.2f}, Test Accuracy: {:.2f}%'.format(loss, acc*100))
print(model.summary())


2/2 [==============================] - 7s 1s/step - loss: 1.5119 - accuracy: 0.5250
Test Loss: 1.51, Test Accuracy: 52.50%
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_19 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 112, 112, 64)     0         
 2D)                                                             
                                                                 
 conv2d_20 (Conv2D)          (None, 112, 112, 128)     73856     
                                                                 
 conv2d_21 (Conv2D)          (None, 112, 112, 128)     147584    
                                                               

# Transfer Learning using VGG19

In [27]:
# load the pre-trained VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# create a new model by adding layers on top of the base model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# freeze the weights of the base model
base_model.trainable = False

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
t1 = time.time()
model.fit(train_data, train_labels,batch_size=batch_size,epochs=num_epochs,validation_data=(val_data, val_labels))
t2 = time.time()
print('Time take in training:',t2-t1,'seconds')

4/4 [==============================] - 50s 13s/step - loss: 8.2277 - accuracy: 0.7266 - val_loss: 3.7094 - val_accuracy: 0.8750
Time take in training: 50.286916732788086 seconds


In [28]:
# evaluate the model on the test data
loss, acc = model.evaluate(test_data, test_labels)
print('Test Loss: {:.2f}, Test Accuracy: {:.2f}%'.format(loss, acc*100))
print(model.summary())

2/2 [==============================] - 16s 6s/step - loss: 9.5902 - accuracy: 0.8750
Test Loss: 9.59, Test Accuracy: 87.50%
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_5 (Flatten)         (None, 25088)             0         
                                                                 
 dense_10 (Dense)            (None, 128)               3211392   
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                                 
Total params: 23,235,905
Trainable params: 3,211,521
Non-trainable params: 20,024,384
_________________________________________________________________
None


# MLP with 55,849,217 parameters (approx. equal to parameters in above model VGG19)


In [37]:
from tensorflow.keras import Sequential, layers

model = Sequential()

# add input layer
model.add(layers.Flatten(input_shape=(224, 224, 3)))

# add hidden layers with 64 neurons each
model.add(Dense(256, activation='relu'))
for i in range(5):
    model.add(Dense(2048, activation='relu'))

# add output layer
model.add(Dense(1, activation='softmax'))


In [38]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
t1 = time.time()
model.fit(train_data, train_labels,batch_size=batch_size,epochs=num_epochs,validation_data=(val_data, val_labels))
t2 = time.time()
print('Time take in training:',t2-t1,'seconds')

Epoch 1/2
4/4 [==============================] - 5s 856ms/step - loss: 32216.5059 - accuracy: 0.5000 - val_loss: 17.4180 - val_accuracy: 0.4688
Epoch 2/2
4/4 [==============================] - 3s 751ms/step - loss: 207.7239 - accuracy: 0.5000 - val_loss: 100.8594 - val_accuracy: 0.4688
Time take in training: 8.347556829452515 seconds


In [39]:
# evaluate the model on the test data
loss, acc = model.evaluate(test_data, test_labels)
print('Test Loss: {:.2f}, Test Accuracy: {:.2f}%'.format(loss, acc*100))
print(model.summary())

2/2 [==============================] - 0s 45ms/step - loss: 128.6171 - accuracy: 0.5250
Test Loss: 128.62, Test Accuracy: 52.50%
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_10 (Flatten)        (None, 150528)            0         
                                                                 
 dense_81 (Dense)            (None, 256)               38535424  
                                                                 
 dense_82 (Dense)            (None, 2048)              526336    
                                                                 
 dense_83 (Dense)            (None, 2048)              4196352   
                                                                 
 dense_84 (Dense)            (None, 2048)              4196352   
                                                                 
 dense_85 (Dense)            (None, 2048)              4